## Animating the action of the Arnold map on a discrete image

The Arnold's cat map is a chaotic map acting on the torus $\mathbb{T}^2= \mathbb{R}^2/\mathbb{Z}^2$, represented by the unit square,  $[0,1) \times [0,1)$.

This  map is defined by $f(x,y) =(x', y')$, with:

$$\left(\begin{array}{l}x'\\y'\end{array}\right) =\left(\begin{array}{ll}2&1\\1&1\end{array}\right)\left(\begin{array}{l}x\\y\end{array}\right) \:\:(\mbox{mod}\: 1)$$ 

or by $$\left(\begin{array}{l}x'\\y'\end{array}\right) =\left(\begin{array}{ll}1&1\\1&2\end{array}\right)\left(\begin{array}{l}x\\y\end{array}\right) \:\:(\mbox{mod}\: 1)$$ 

The two maps are  equivalent (topologically conjugate), i.e. they exhibit the same dynamical behaviour.   The first version is mostly used in the study of chaotic dynamical systems, while the second in connection to image encryption.

The discrete version, $f_d$,  of the Arnold map acts on a  grid over the unit square or on an image of resolution $N \times N$,
represented by an array of shape (N,N).
This map is defined as follows:
$$f_d : \{0,1,2, \ldots, N-1\} \times  \{0,1,2, \ldots, N-1\} \to  \{0,1,2, \ldots, N-1\} \times  \{0,1,2, \ldots, N-1\}, \quad f_d(x, y)=(x', y'),$$
with:
  $$\left(\begin{array}{l}x'\\y'\end{array}\right) =\left(\begin{array}{ll}2&1\\1&1\end{array}\right)\left(\begin{array}{l}x\\y\end{array}\right), \:\:(\mbox{mod}\: N)$$ 
  
  Here (x, y) are the coordinates of a pixel in an image referenced to a right system of axes, Ox, Oy, with origin, O, coinciding with the lower left pixel. If (i, j) are the (row, column) of a pixel, and (x,y) the coordinates of the same pixel, then j=x, and i=N-1-y.
  If Img is a fixed image, and Img' = f_d(Img) is the resulted image through the transformation  $f_d$, then to a  pixel in Img, of coordinates (x,y), and color c,   corresponds the pixel of coordinates (x', y') and the same color in Img'.  Hence the discrete Arnold map shuffles the pixel colors. After  some iteration the  mixing of colors occurs.
  
  On the other hand the discrete version of the Arnold map has a regularity that is not exhibited by the classical Arnold map, that acts on the entire torus, not only on a grid on it. Namely,  there exists a  positive integer, p(N), that depends on the image resolution,  such that $f^p_d(Img)=Img$, i.e after a number of iterations we get again the original image. The  least number, p(N), with this property is called the image period with respect to the map $f_d$. The period p(N) or an upper bound for it   is deduced in: [https://arxiv.org/abs/1111.2984](https://arxiv.org/abs/1111.2984)), [https://link.springer.com/article/10.1007/s11071-012-0539-3](https://link.springer.com/article/10.1007/s11071-012-0539-3)).
  
Note that the periodicity of all  images of the same resolution, $N\times N$, with respect to the Arnold map $f_d$ is a consequence of the property $A^p=Id$, where
$A$  is the matrix in the Arnold map definition, and the product $A^p$ is computed in $Z_N$, i.e. modulo N.

For image encryption algorithms periodicity of an image is a drawback. That's why generalized Arnold maps are used instead of the classical Arnold maps, defined above. These maps  depend on two parameters that can be adjusted to get a map that ensures a bigger period for an image of resolution $N\times N$. 

 A generalized Arnold map   is represented by an integer matrix of the form:
  
  $$A=\left(\begin{array}{ll}1+ab&a\\b&1\end{array}\right)\:\: \mbox{or}\:\: A= \left(\begin{array}{ll}1&a\\b&1+ab\end{array}\right), \:\: a, b, \mbox{positive integers}$$  
  For a=b=1 we get the classical versions of the Arnold map.


The aim of this notebook is twofold: first, to illustrate the properties of the discrete Arnold cat map, and second to give an example of 
    simultaneous 2d and 3d plot animations, as well as to perform the layout update for each frame. 

To illustrate how the discrete Arnold map acts on an image we define a subplot with two windows. In the left one we are displaying a cat image, while in the right one the image mapped, as a texture,  onto a 2D toral surface. Then we are applying succesive iteration of the discrete Arnold map to the original image, and plot
as a new animation frame the corresponding planar, respectively toral image.

In [ ]:
import numpy as np
from numpy import sin, cos, pi
import skimage.io as sio
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
url='https://raw.githubusercontent.com/empet/Discrete-Arnold-map/master/Images/cat-128.jpg'
imag = sio.imread (url)
img = imag[:, :, 1]
img.shape

An image of resolution $128\times 128$ has the period 96,  with respect to the classical Arnold map (a=b=1).

In [ ]:
N = img.shape[0]
surfcolor = [np.flipud(img)]
#Parameterization of the 2d torus (as a surface)
su = np.linspace(-0.5, 0.5, N)
sv = np.linspace(0, 1, N)
u, v = np.meshgrid(su,sv)
R = 4
r = 1
x = (R + r*cos(2*pi*v)) * cos(2*pi*u)
y = (R + r*cos(2*pi*v)) * sin(2*pi*u)
z = r * sin(2*pi*v) 

Define a function that returns the list of images obtained by iterating an Arnold map n_iterations times, i.e. an image orbit:

In [ ]:
def map_iterate(img,  a=1, b=1, n_iterations=1):
    m, n = img.shape
    if m != n:
        raise ValueError('Arnold map acts on images of shape N x N')
    imagelist = [np.flipud(img)]
    idx = np.arange(m)
    xim, yim = np.meshgrid(idx, idx) #(xim, yim) are the pixel  (x,y)-coordinates 
    for _ in range(1, n_iterations):
        xtemp = ((1+a*b)*xim + a*yim) % m
        yim = (b*xim + yim) % m
        xim = xtemp
        imagelist.append(np.flipud(img[yim, xim]))                                        
    return imagelist  


Define 6 frames for animation: the first one displays the original images, and the others the images resulted from iterating
the  Arnold map. After the  $5^{th}$ iteration the colors are mixed.

In [ ]:
nframes = 6
surfcolor = map_iterate(img, n_iterations=nframes)

Define the subplot figure:

In [ ]:
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "xy"}, {"type": "scene"}]],
    horizontal_spacing=0.05)

fig.add_trace(go.Heatmap(z = surfcolor[0], 
                         coloraxis='coloraxis'), 1, 1)

fig.add_trace(go.Surface(x=x, y=y, z=z,
                         surfacecolor=surfcolor[0], 
                         coloraxis='coloraxis' ))

axes3D = dict(scene_xaxis_visible=False, 
              scene_yaxis_visible=False, 
              scene_zaxis_visible=False)

fig.update_layout(title_text = f'Left: image of resolution 128 x128. Right: image mapped on the torus',
                  title_x=0.5, font_color='white',
                  width=700, height=450, scene_aspectmode='data',
                  coloraxis=dict(colorscale='matter_r', 
                                 showscale=False,
                                 ),
                  xaxis_showticklabels=False,
                  yaxis_showticklabels=False,
                  **axes3D,
                  paper_bgcolor='black');

In [ ]:
#fig.show()

Define the animation frames:

In [ ]:
frames = [ ]

for k in range(nframes):
    frames.append(go.Frame(data= [go.Heatmap(z=surfcolor[k]),
                                  go.Surface(surfacecolor=surfcolor[k])],
                           layout= go.Layout(title_text=f'Arnold cat map, iteration #{k}'),
                           traces =[0,1]))
    
fig.update_layout(updatemenus=[dict(type='buttons',
                                  showactive=False,
                                  y=0,
                                  x=1.05,
                                  xanchor='left',
                                  yanchor='top',
                                  pad=dict(t=1),
                                  buttons=[dict(label='Play',
                                                method='animate',
                                                args=[None, dict(frame=dict(duration=50, redraw=True), 
                                                                 transition=dict(duration=0),
                                                                 fromcurrent=True,
                                                                 mode='immediate'
                                                                 )])
                                          ]
                                  )])   
fig.update(frames=frames);


In [2]:
%%html 
<img src="anim-arnold-cat.gif">

Let us illustrate the periodicity of the (cat) image (cropped to get the resolution $124 \times 124$), with respect to the generalized Arnold map, defined by a=40, and b=8.  Following the results in the above mentioned references, we deduced that an image of such resolution has the period 5. Let us confirm it through simulation:

In [ ]:
#imag = sio.imread ("cat-128.jpg")  
img1 = imag[2:-2, 2:-2, 1]
img1.shape

In [ ]:
imagelist = map_iterate(img1, a=40, b=8, n_iterations=6)

In [ ]:
fig124 = make_subplots(
    rows=2, cols=3,
    horizontal_spacing=0.02,
    vertical_spacing=0.05,
    subplot_titles=('Original image', 'Iteration #1', 'Iteration #2', 'Iteration #3', 'Iteration #4', 'Iteration #5')
)

for k in range(6):
    fig124.add_trace(go.Heatmap(z=imagelist[k], coloraxis='coloraxis'), k//3+1, k-3*(k//3)+1)
    fig124.update_layout(width=800, height=600,
                         title_text='5-periodic image with respect to the action of a generalized Arnold map',
                         title_x=0.5,
                        coloraxis=dict(colorscale='matter_r', colorbar_thickness=25))
    fig124.update_xaxes(showticklabels=False, ticks='')
    fig124.update_yaxes(showticklabels=False, ticks='')


In [ ]:
fig.show()

.

.